In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns  #data visulization
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#import datasets
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
train.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
#checking null values
sns.heatmap(train.isnull(),cmap='viridis')

In [ ]:
sns.heatmap(test.isnull(),cmap='viridis')

In [ ]:
train['target'].unique()

In [ ]:
X = train.drop(['id','target'],axis=1)
y = train['target']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
y

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 100)

In [ ]:
print(X_train.shape)
print(X_test.shape)

# KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
knn_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,knn_pred))
print('\n')
print(classification_report(y_test,knn_pred))
print('\n')

print(f'Accuracy: {accuracy_score(y_test,knn_pred)*100:.2f}%')

In [ ]:
#to find the k value, finding the error each time the model predicts between k value 1 to 100
error = []

for i in range(1,100):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred = knn.predict(X_test)
    error.append(np.mean(pred != y_test))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(range(1,100),error,color='red',marker='o',markerfacecolor='blue',markersize=4)
plt.xlabel('K')
plt.ylabel('Error')


In [ ]:
#choosing k value = 70
knn = KNeighborsClassifier(n_neighbors=80)
knn.fit(X_train,y_train)
knn_pred = knn.predict(X_test)

print(confusion_matrix(y_test,knn_pred))
print(classification_report(y_test,knn_pred))
                            

In [ ]:
print(f'Accuracy: {accuracy_score(y_test,knn_pred)*100:.2f}%')

# GridSearchCV - KNN

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#model training with whole train data set
param_grid = {'n_neighbors':list(range(85,100))} 
grid = GridSearchCV(KNeighborsClassifier(n_neighbors=30),param_grid,refit=True,verbose=2)
grid.fit(X,y)

In [ ]:
print(grid.best_score_)
print(grid.best_estimator_)

In [ ]:
prediction = grid.predict_proba(test.drop('id',axis=1))

In [ ]:
prediction

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.1)
gb.fit(X_train,y_train)
y_pred = gb.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
                            

In [ ]:
gb.fit(X,y)
prediction = gb.predict_proba(test.drop('id',axis=1))

In [ ]:
prediction

# Submission

In [ ]:
submission = pd.DataFrame({'id':test['id'],'Class_1': prediction[:,0:1].flatten(),'Class_2':prediction[:,1:2].flatten(),
                           'Class_3':prediction[:,2:3].flatten(),'Class_4':prediction[:,3:].flatten()})
submission.to_csv(f'submission.csv',index=False)

In [ ]:
submission